<a href="https://colab.research.google.com/github/dominikklepl/News-search-engine/blob/master/News_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load the dataset

In [5]:
PATH = "drive/My Drive/news_classification.json"
data = pd.read_json(PATH, lines=True)
data.head(5)

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


Drop some columns as we don't need those

In [6]:
data = data.drop(['authors', 'link', 'date'], axis=1)
data.head(5)

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."


Merge the text columns into one since extracting features will be easier this way

In [7]:
data['text'] = data['headline'] + " " + data['short_description']
data.head(5)

,category,headline,short_description,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...


And again we can drop now useless columns: headline and short_description

In [8]:
data = data.drop(['headline', 'short_description'], axis=1)
data.head(5)

,category,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


Let's look at number of news per category

In [9]:
categories = data.groupby('category')
print("There's {} news categories." .format(categories.ngroups))
print(categories.size())

There's 41 news categories.
category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WOR

There are some categories that could be treated as one. For example ARTS & CULTURE, ARTS and CULTURE & ARTS. We're not interested in some of the categories so we'll drop those.

In [0]:
culture = ['ARTS & CULTURE', 'ARTS', 'CULTURE & ARTS']
home_living = ['HOME & LIVING', 'HEALTHY LIVING', 'WELLNESS']
style = ['STYLE', 'STYLE & BEAUTY']
entertainment = ['COMEDY', 'ENTERTAINMENT', 'MEDIA']
business = ['MONEY', 'BUSINESS']
parenting = ['PARENTING', 'PARENTS']
science_tech = ['SCIENCE', 'TECH']
education = ['COLLEGE', 'EDUCATION']
drop = ['BLACK VOICES', 'DIVORCE', 'FIFTY', 'GOOD NEWS', 'IMPACT', 'LATINO VOICES', 'WOMEN', 
        'QUEER VOICES', 'TASTE', 'THE WORLDPOST', 'WORLD NEWS', 'WORLDPOST', 'GREEN', 'WEIRD NEWS', 'DIVORCE', ]

In [0]:
data_improved = data[~data.category.isin(drop)]

In [12]:
data_improved.category[data_improved.category.isin(culture)] = "CULTURE"
data_improved.category[data_improved.category.isin(home_living)] = "HOME & LIVING"
data_improved.category[data_improved.category.isin(style)] = "STYLE"
data_improved.category[data_improved.category.isin(entertainment)] = "ENTERTAINMENT"
data_improved.category[data_improved.category.isin(business)] = "BUSINESS"
data_improved.category[data_improved.category.isin(parenting)] = "PARENTING"
data_improved.category[data_improved.category.isin(education)] = "EDUCATION"
data_improved.category[data_improved.category.isin(science_tech)] = "SCIENCE & TECH"

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Reset index

In [0]:
data_improved = data_improved.reset_index(drop=True)

In [14]:
len(data_improved)

159900

Remove digits as they are rarely indicative of topic. Except maybe business has more numbers but that probably wouldn't help much.

In [0]:
data_improved.text = data_improved.text.replace('[0-9]', '', regex=True)

> Compute text lenght (number of words) and exclude very short ones.

In [0]:
lenght = []
for i in range(len(data_improved)):
    lenght.append(len(data_improved.text[i].split()))

In [0]:
data_improved['lenght'] = lenght

In [19]:
data_improved.head(5)

,category,text,lenght
0,CRIME,"There Were Mass Shootings In Texas Last Week,...",25
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The ...,19
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age ...,24
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,25
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,26


In [20]:
data_improved = data_improved[data_improved.lenght>10]
len(data_improved)

148549

In [21]:
categories = data_improved.groupby('category')
print("There's {} news categories." .format(categories.ngroups))
print(categories.size())

There's 16 news categories.
category
BUSINESS           6984
CRIME              2839
CULTURE            3472
EDUCATION          1928
ENTERTAINMENT     20887
ENVIRONMENT        1320
FOOD & DRINK       6082
HOME & LIVING     27538
PARENTING         12321
POLITICS          30458
RELIGION           2030
SCIENCE & TECH     3897
SPORTS             4339
STYLE             11293
TRAVEL             9512
WEDDINGS           3649
dtype: int64


### Remove stopwords and lemmatize

Can placed in the vectorizer function

In [0]:
import spacy
from html import unescape

# create a spaCy tokenizer
spacy.load('en')
lemmatizer = spacy.lang.en.English()

#to lower, remove HTML tags
def my_preprocessor(doc):
    return(unescape(doc).lower())

# tokenize the doc and lemmatize its tokens
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])

### Train-test split
Keep 10% aside for testing

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_improved.text, data_improved.category, test_size=0.2, random_state=42)

### K-fold split

In [0]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 1)

## BoW and Tfidf model
Let's start with just a simple bag of words and tfidf features. We'll use both 1- and 2-grams for calculating these features. The number of features is limited to 15k so that only the most useful features are extracted.

In [0]:
vect = TfidfVectorizer(stop_words='english', ngram_range = (1,2), max_features = 20000, preprocessor=my_preprocessor, tokenizer=my_tokenizer)

#### MODELS
* Naive Bayes
* Ridge
* XGBoost

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from xgboost.sklearn import XGBClassifier

In [0]:
NB = MultinomialNB()
Ridge = RidgeClassifier()
XGB = XGBClassifier(objective = 'multi:softprob')

In [0]:
def fit_cv(clf , name):
    pipe = Pipeline([
    ('vectorize', vect),
    (name, clf)])
    result = cross_validate(pipe, X_train, y_train, cv = folds, return_train_score=True,scoring = ('accuracy', 
                                                                                       'f1_weighted', 
                                                                                       'precision_weighted', 
                                                                                       'recall_weighted'))
    return result

In [0]:
bayes = fit_cv(NB, 'NB')
ridge = fit_cv(Ridge, 'Ridge')
xgb = fit_cv(XGB, 'XGB')

### Evaluate models

In [0]:
bayes.keys()

In [0]:
b = pd.DataFrame.from_dict(bayes)
b['model'] = ["NB","NB","NB","NB","NB"]
r = pd.DataFrame.from_dict(ridge)
r['model'] = ["Ridge","Ridge","Ridge","Ridge","Ridge"]
x = pd.DataFrame.from_dict(xgb)
x['model'] = ["XGB","XGB","XGB","XGB","XGB"]

results = pd.concat([b,r,x])

In [0]:
results.sample(6)

In [0]:
models_eval = results.groupby("model")

In [0]:
means = models_eval.mean()
sd = models_eval.std()

In [0]:
means.head()

In [0]:
sd.head()

In [0]:
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_curve, auc 


def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        


def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  labels=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                labels=level_labels)) 
    print(cm_frame) 
    
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)
    
    
    
def model_perf(true_labels, predicted_labels, classes=y_test.unique()):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)

## Fit best model on whole training dataset

In [88]:
Ridge = RidgeClassifier()
pipe = Pipeline([('vectorize', vect),
                 ("model", Ridge)])
final_m = pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['3', 'far', 'good', 'little', 'make', '\ufeff1'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


### Final evaluation on testing dataset

In [0]:
y_pred = final_m.predict(X_test)

In [0]:
model_perf(y_test, y_pred)

### Train on full dataset

In [0]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

Ridge = RidgeClassifier()
pipe = Pipeline([('vectorize', vect),
                 ("model", Ridge)])
final_m = pipe.fit(X_train, y_train)

#### Save the trained model

In [80]:
from joblib import dump, load
dump(final_m, 'news_classifier.joblib') 

['news_classifier.joblib']